yfinance API Documentation: https://ranaroussi.github.io/yfinance/reference/index.html 

git repo: https://github.com/tomwmoore/tm-finance-lab

In [10]:
import sys 
import os

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

import importlib


# Add root path so other subfolders are accessible
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


import src.utils.db_azure
# Reload it every time the cell is run
importlib.reload(src.utils.db_azure)

from src.utils.db_azure import get_analytics_azure_engine
from src.utils.db_azure import get_dashboard_azure_engine
from src.utils.db_azure import azure_upsert

# Custom Plots
import src.utils.plots
importlib.reload(src.utils.plots) # Force reload every time the cell is run
from src.utils.plots import plot_stock

# Custom Indicators
import src.utils.indicators
importlib.reload(src.utils.indicators) # Force reload every time the cell is run
from src.utils.indicators import compute_rsi


# Helper Functions
import src.utils.yfinance_helpers
importlib.reload(src.utils.yfinance_helpers) # Force reload every time the cell is run
from src.utils.yfinance_helpers import *


In [ ]:
    # Get list of tickers from given industry from asset header
query_symbol = f"""
                select symbol 
                from asset_header
                where industry = 'Oil & Gas E&P'
                """

engine = get_analytics_azure_engine()

symbols = pd.read_sql(query_symbol,engine)
symbols['symbol'].tolist()

In [7]:
# get some stock data
# tickers = symbols['symbol'].tolist()
tickers = 'CL=F'

# data = yf.download(ticker, start="2025-01-01", end="2025-09-25")
data = yf.download(tickers, interval= '1d', group_by='ticker', start = '2024-01-01', end = '2025-01-01' )

# Clean up multi-index column names to just be the feature names (close, open etc) & make lowercase
data = data.stack(level=0,future_stack=True).rename_axis(['Date', 'Ticker']).reset_index(level=1)

# remote date as the index
data = data.reset_index()

data.columns = [col.lower() for col in data.columns]

# add current timestamp for future QC
data['updated_at'] = pd.Timestamp.now()




C:\Users\tomwm\AppData\Local\Temp\ipykernel_5964\1986841399.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [15]:
test = get_sector(symbol='CL=F')

None


In [11]:
from sqlalchemy.dialects.mssql import DATETIME2, VARCHAR, FLOAT, BIGINT

# Create new table and load to sql
azure_engine = get_analytics_azure_engine()


dtype_mapping = {
    'date': DATETIME2(6),        # precise for milliseconds
    'ticker': VARCHAR(20),       
    'open': FLOAT,
    'high': FLOAT,
    'low': FLOAT,
    'close': FLOAT,
    'volume': BIGINT,            
    'updated_at': DATETIME2(0)   # drops seconds, rounds to nearest minute
}

data.to_sql(
    'stock_prices',
    azure_engine,
    if_exists='fail', 
    index=False,
    dtype=dtype_mapping
)

165

In [14]:
# Create indices on new table
from sqlalchemy import text

with azure_engine.begin() as conn:
    conn.execute(text('alter table stock_prices alter column date datetime2 NOT NULL'))
    conn.execute(text('alter table stock_prices alter column ticker varchar(20) NOT NULL'))
    conn.execute(text('alter table stock_prices add constraint pkey_stock_prices PRIMARY KEY (date,ticker)'))

In [24]:
# upsert dataframe to postgres
azure_engine = get_analytics_azure_engine()
azure_upsert(data,azure_engine,'stock_prices')

In [5]:
# get data from postgres db
symbol = 'TVE.TO'
azure_engine = get_analytics_azure_engine()
select_query = f"select * from stock_prices where symbol = '{symbol}'"

df = pd.read_sql(select_query,azure_engine)

df.head()

,date,symbol,open,high,low,close,volume,updated_at
0,2020-01-02,TVE.TO,1.742394,1.759905,1.707371,1.733638,345600,2025-10-08 22:14:16
1,2020-01-03,TVE.TO,1.759905,1.803684,1.724882,1.724882,592100,2025-10-08 22:14:16
2,2020-01-06,TVE.TO,1.742395,1.926265,1.742395,1.908754,2186500,2025-10-08 22:14:16
3,2020-01-07,TVE.TO,1.891242,1.987555,1.856219,1.961288,951900,2025-10-08 22:14:16
4,2020-01-08,TVE.TO,1.978799,1.978799,1.777417,1.847463,1177400,2025-10-08 22:14:16


In [6]:
# Plot prices & custom indicators
import src.utils.plots
# Reload it every time the cell is run
importlib.reload(src.utils.plots)

from src.utils.plots import plot_stock

import src.utils.indicators
# Reload it every time the cell is run
importlib.reload(src.utils.indicators)

from src.utils.indicators import compute_rsi



df_plot = df.copy()

target_period = 15

df_plot = df_plot[df_plot['date'] > '2025-01-01']

df_plot['rolling_avg'] = df_plot['close'].rolling(window=target_period).mean()
df_plot['upper_band'] = df_plot['rolling_avg'] + 2*df_plot['close'].rolling(window=target_period).std()
df_plot['lower_band'] = df_plot['rolling_avg'] - 2*df_plot['close'].rolling(window=target_period).std()

df_plot['rsi'] = compute_rsi(df_plot['close'],period=target_period)
 
fig = plot_stock(df_plot, 
                 date_col='date', 
                 price_col='close',
                 rsi = df_plot['rsi'],
                 plot_volume= True,
                 indicators= {
                    f'Rolling Avg ({target_period}d)': {'data': df_plot['rolling_avg'], 'color': 'red'},
                    'Upper Band': {'data': df_plot['upper_band'], 'color': 'green', 'dash': 'dot'},
                    'Lower Band': {'data': df_plot['lower_band'], 'color': 'green', 'dash': 'dot'}
                })
fig.show()